# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

city_data_df = city_data_df.dropna()

# Reset the index
city_data_df = city_data_df.reset_index(drop=True)

# Convert 'Humidity' to numeric, coercing errors
city_data_df['Humidity'] = pd.to_numeric(city_data_df['Humidity'], errors='coerce')

# Remove rows with NaN 'Humidity'
city_data_df = city_data_df[~city_data_df['Humidity'].isnull()]

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0            colonia  50.9333    6.9500     22.36        67   
1        1           westport  41.1415  -73.3579     31.72        47   
2        2  port-aux-francais -49.3500   70.2167      6.25        82   
3        3           la sarre  48.8002  -79.1996     26.15        35   
4        4         whitehorse  60.7161 -135.0538     12.43       100   

   Cloudiness  Wind Speed Country        Date  
0         100        5.66      DE  1690135484  
1           0        3.58      US  1690135289  
2          81       17.69      TF  1690135485  
3          24        3.09      CA  1690135485  
4         100        1.03      CA  1690135485

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat',
                             geo=True, # Enable geographic coordinates
                             color='Humidity', # Color points by humidity
                             size='Humidity', # Size points by humidity
                             hover_cols=["City"], # Show city name on hover
                             title='City Weather Data'
                             )

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values

ideal_cities = city_data_df[(city_data_df["Max Temp"] >= 21) & 
                            (city_data_df["Max Temp"] <= 27) & 
                            (city_data_df["Wind Speed"] <= 4.5) & 
                            (city_data_df["Cloudiness"] == 0)].dropna()
# Display ideal cities  
ideal_cities

City_ID           City      Lat       Lng  Max Temp  Humidity  \
44        45        malanje  -9.5402   16.3410     24.86        22   
242      243         shingu  33.7333  135.9833     23.08        91   
267      268    chibougamau  49.9168  -74.3659     23.79        33   
323      324       fairwood  47.4484 -122.1573     24.82        58   
340      341          beira -19.8436   34.8389     21.05       100   
423      426  fort mcmurray  56.7268 -111.3810     21.73        73   
542      546       tongshan  34.1805  117.1571     26.52        77   
546      550        ilabaya -17.4208  -70.5133     25.20        27   
550      554   saint-pierre -21.3393   55.4781     21.98        78   

     Cloudiness  Wind Speed Country        Date  
44            0        2.14      AO  1690135494  
242           0        1.57      JP  1690135525  
267           0        3.22      CA  1690135357  
323           0        3.60      US  1690135537  
340           0        2.68      MZ  1690135519  
423           0        2.57      CA  1690135568  
542           0        1.71      CN  1690135586  
546           0        4.05      PE  1690135586  
550           0        1.54      RE  1690135587

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
44         malanje      AO  -9.5402   16.3410        22           
242         shingu      JP  33.7333  135.9833        91           
267    chibougamau      CA  49.9168  -74.3659        33           
323       fairwood      US  47.4484 -122.1573        58           
340          beira      MZ -19.8436   34.8389       100           
423  fort mcmurray      CA  56.7268 -111.3810        73           
542       tongshan      CN  34.1805  117.1571        77           
546        ilabaya      PE -17.4208  -70.5133        27           
550   saint-pierre      RE -21.3393   55.4781        78

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
api_key = geoapify_key

# Print a message to indicate the hotel search is starting
print("Starting hotel search")

# Create an empty list to store hotel data
hotel_data = pd.DataFrame(columns=["City", "Place Name", "Place Latitude", "Place Longitude"])


# Iterate through the ideal cities DataFrame
for index, row in ideal_cities.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Construct the URL with the current city's latitude, longitude, and radius
    #url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:{latitude},{latitude},{longitude},{longitude}&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783"
    url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{longitude},{latitude},10000&bias=proximity:{longitude},{latitude}&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783"
    print(f"URL: {url}")  # Print the URL

    # Make an API request using the constructed URL
    response = requests.get(url)

    # Convert the API response to JSON format
    response_data = response.json()

   #print(f"Response: {response_data}")  # Print the response

    # Check if any places were found
    if response_data.get("features"):
        # Iterate through the place features and extract relevant information
        for feature in response_data["features"]:
            properties = feature.get("properties")
            if properties and "name" in properties:
                place_name = properties["name"]
                place_lat = feature["geometry"]["coordinates"][1]
                place_lon = feature["geometry"]["coordinates"][0]
                
                new_row = pd.DataFrame({
                "City": [row['City']],
                "Place Name": [place_name],
                "Place Latitude": [place_lat],
                "Place Longitude": [place_lon]
                })

                hotel_data = pd.concat([hotel_data, new_row], ignore_index=True)
   
    else:
        print("Invalid data format for a place. Skipping...")

# Clean hotel data
hotel_data = hotel_data.dropna()

# Reset the index
hotel_data = hotel_data.reset_index(drop=True)

# Create a DataFrame from the hotel data

hotel_df = pd.DataFrame(hotel_data)


# Display the hotel DataFrame
print(hotel_df)


Starting hotel search
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:16.341,-9.5402,10000&bias=proximity:16.341,-9.5402&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:135.9833,33.7333,10000&bias=proximity:135.9833,33.7333&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-74.3659,49.9168,10000&bias=proximity:-74.3659,49.9168&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
Invalid data format for a place. Skipping...
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-122.1573,47.4484,10000&bias=proximity:-122.1573,47.4484&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:34.8389,-19.8436,10000&bias=proximity:34.8389,-19.8436&limit=20&apiKey=d548c5ed24604be6a9dd0d9

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display



# Configure the map plot
map_plot = hotel_df.hvplot.points('Place Longitude', 'Place Latitude',
                                  geo=True,  # Enable geographic coordinates
                                  hover_cols=["City", "Place Name", "Country"],  # Show city name, hotel name, and country on hover
                                  title='City Weather Data'
                                 )

# Display the map
map_plot 

:Points   [Place Longitude,Place Latitude]   (City,Place Name)